<a href="https://colab.research.google.com/github/chrishg23-jpg/HES-benchmark/blob/main/HES2full5check5_105.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# HES_23.0_RESONANCE_SWEEP.py: Mapping Stability vs. Grid Size (N=5 to N=105)

import numpy as np
import time
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
np.random.seed(42)

# --- FIXED OPTIMAL PARAMETERS ---
ALPHA = 3.00
RHO_OPTIMAL = 1.647
DELTA = ALPHA * RHO_OPTIMAL  # 4.941
BETA = 0.1
GAMMA = 0.001

# --- SWEEP PARAMETERS ---
N_START, N_END = 5, 105
N_RANGE = range(N_START, N_END + 1)
STEPS = 50
SUBSYSTEM_SIZE = 3
THRESHOLD_G_MAX = 1.0e+10
THRESHOLD_LAMBDA_MIN = 0.0

# --- CORE SIMULATION FUNCTIONS (Same full logic as HES 22.0) ---
# ... (von_neumann_entropy, laplacian_2d_numpy functions are included here) ...

def von_neumann_entropy(subsystem):
    """Computes von Neumann entropy of a subsystem's density matrix."""
    flat = subsystem.flatten()
    rho = np.outer(flat, flat)
    rho /= np.trace(rho + 1e-10)
    eigenvalues = np.linalg.eigvalsh(rho)
    eigenvalues = eigenvalues[eigenvalues > 1e-10]
    return -np.sum(eigenvalues * np.log2(eigenvalues)) if len(eigenvalues) > 0 else 0.0

def laplacian_2d_numpy(field):
    """OPTIMIZED Laplacian (∇²S)."""
    S_padded = np.pad(field, 1, mode='wrap')
    S_center = S_padded[1:-1, 1:-1]
    L = S_padded[2:, 1:-1] + S_padded[:-2, 1:-1] + S_padded[1:-1, :-2] + S_padded[1:-1, 2:] - 4 * S_center
    return L

# --- RUN SIMULATION MODIFIED FOR SWEEP ---
def run_diagnostic(N, alpha, delta, beta, gamma):
    """Runs HES simulation for a specific N and returns stability metrics."""
    L_SQUARE = (N / 100.0)**2

    s = np.random.uniform(-1, 1, (N, N))
    entropy_map = np.zeros((N, N))
    C_history = []

    for step in range(STEPS):
        # 1. Compute Entropy Map
        for i in range(N):
            for j in range(N):
                i_start, i_end = max(0, i - SUBSYSTEM_SIZE//2), min(N, i + SUBSYSTEM_SIZE//2 + 1)
                j_start, j_end = max(0, j - SUBSYSTEM_SIZE//2), min(N, j + SUBSYSTEM_SIZE//2 + 1)
                if i_end > i_start and j_end > j_start:
                    entropy_map[i,j] = von_neumann_entropy(s[i_start:i_end, j_start:j_end])

        # 2. Compute Curvature and APPLY CORRECT L^2 NORMALIZATION
        curvature = laplacian_2d_numpy(entropy_map)
        curvature_norm = curvature / L_SQUARE

        # 3. Update Rule (Fixed Beta/Gamma)
        nabla_s_approx = (s - (np.roll(s, 1, axis=0) + np.roll(s, -1, axis=0) +
                               np.roll(s, 1, axis=1) + np.roll(s, -1, axis=1)) / 4.0)
        s_new = s - (alpha * nabla_s_approx + beta * s + gamma * np.mean(s) * s + delta * curvature_norm)
        s = np.clip(s_new, -1, 1)
        C_history.append(np.mean(np.abs(curvature_norm)))

    # --- POST-RUN METRICS ---

    # Estimate G_eff
    boundary_area = 4 * N
    boundary_entropy = np.mean([entropy_map[0,:], entropy_map[-1,:],
                                entropy_map[:,0], entropy_map[:,-1]])
    G_eff = boundary_area / (4 * boundary_entropy) if boundary_entropy > 0 else 1.0e+20

    # Compute Lambda (λ_sim)
    lambda_sim = 0.0
    FIT_START, FIT_END = 20, 50
    C_data = np.array(C_history[FIT_START:FIT_END])
    if C_data.size >= 2 and np.max(C_data) > 1e-18:
        log_C = np.log(C_data + 1e-18)
        time_steps = np.arange(len(log_C))
        poly_coeffs = np.polyfit(time_steps, log_C, 1)
        lambda_sim = -poly_coeffs[0]

    return lambda_sim, G_eff


# --- MAIN SWEEP LOOP ---
def perform_resonance_sweep():
    survivor_grids = []
    total_runs = len(N_RANGE)
    start_time = time.time()

    print(f"Starting HES Resonance Sweep: N={N_START} to N={N_END} ({total_runs} runs).")

    for N in N_RANGE:
        # Run the simulation for the current grid size N
        lambda_sim, G_eff = run_diagnostic(N, ALPHA, DELTA, BETA, GAMMA)

        # Check for "Survivor" (Stable Lock)
        if lambda_sim > THRESHOLD_LAMBDA_MIN and G_eff < THRESHOLD_G_MAX:

            # Log the survivor
            correlator_placeholder = 0.8 # Placeholder for full correlator calculation

            survivor_grids.append({
                'N': N,
                'L2': round((N/100.0)**2, 4),
                'lambda_sim': round(lambda_sim, 4),
                'G_eff': round(G_eff, 3)
            })
            print(f"SURVIVOR FOUND! N={N} (L2={L_SQUARE:.4f}) | λ={lambda_sim:.4f}, G={G_eff:.3f}")

    end_time = time.time()
    print(f"\nResonance Sweep Finished in {end_time - start_time:.2f} seconds.")
    return survivor_grids

# --- EXECUTION ---
survivor_list = perform_resonance_sweep()

print("\n--- ACT XXIII: GRID DIMENSIONAL SURVIVORS ---")
if survivor_list:
    # Print the identified harmonics
    for s in survivor_list:
        print(f"N={s['N']}x{s['N']} | L^2={s['L2']:.4f} | λ_sim={s['lambda_sim']:.4f} | G_eff={s['G_eff']:.3f}")

    #
    print(f"\nTotal Dimensional Harmonics Found: {len(survivor_list)}")
else:
    print("No dimensional harmonics (survivors) found outside of the expected collapse regions.")


Starting HES Resonance Sweep: N=5 to N=105 (101 runs).

Resonance Sweep Finished in 818.95 seconds.

--- ACT XXIII: GRID DIMENSIONAL SURVIVORS ---
No dimensional harmonics (survivors) found outside of the expected collapse regions.
